# Extracción de Datos: Ferreterías en Mérida, Yucatán

## Metodología de Recolección de Datos

Este notebook documenta el proceso de extracción de datos de ferreterías utilizando la API de Google Places. La estrategia empleada consiste en:

1. **Segmentación geográfica**: División del área de estudio en múltiples viewports rectangulares
2. **Búsqueda sistemática**: Consulta de cada viewport para obtener datos completos

**Área de estudio**: Zona Norte-Poniente de Mérida, Yucatán

**Fuente de datos**: Google Places API (New)

In [219]:
import folium
import time
import requests
from dotenv import load_dotenv
import os
import pandas as pd

---

### Función de Creación de Viewports

Esta función genera los viewports en el mapa para dividir el área de estudio en secciones manejables.

**Parámetros:**
- `start_lat, start_lng`: Coordenadas de inicio
- `rows, cols`: Número de filas y columnas de rectángulos
- `delta`: Tamaño de cada viewport
- `start_k`: Identificador inicial
- `color`: Color de visualización

**Propósito**: La API de Google Places tiene limitaciones de área por consulta. Esta segmentación permite una cobertura completa sin perder resultados.

In [220]:
def create_viewports(map, start_lat, start_lng, rows, cols, delta, start_k, color):
    viewports = []
    k = start_k
    
    for i in range(rows):
        for j in range(cols):
            low = [start_lat + i * delta, start_lng + j * delta]
            high = [start_lat + (i + 1) * delta, start_lng + (j + 1) * delta]
            
            folium.Rectangle(
                color = 'black',
                weight = 0.6,
                bounds = [low, high],
                tooltip = k,
                fill = True,
                fill_color = color
            ).add_to(map)
            
            k += 1
            
            if delta < 0:
                viewports.append((tuple(high), tuple(low)))
            else:
                viewports.append((tuple(low), tuple(high)))
    
    return viewports, k

---

### Configuración del Mapa de Búsqueda

Creación de la cuadrícula de viewports que cubre toda el área de estudio. Se definen 7 grupos de viewports para abarcar las zonas deseadas sin hacer busquedas de mas:

- **Azul**: Área principal (70 viewports)
- **Verde**: Extensión este (36 viewports)
- **Rojo**: Extensión noreste (30 viewports)
- **Naranja**: Extensión noroeste (10 viewports)
- **Amarillo**: Extensión sureste (12 viewports)
- **Cyan**: Extensión central-oeste (18 viewports)
- **Negro**: Extensión este adicional (30 viewports)

In [221]:
m = folium.Map(location=[20.9939879883004, -89.62853393602846], min_zoom=12)

rectangles_viewports = []
delta = 0.0092
k = 0

first_lat, first_lng = 20.947641060948502, -89.70360532775601
viewports_1, k = create_viewports(m, first_lat, first_lng, 10, 7, delta, k, 'blue')
rectangles_viewports.extend(viewports_1)

sec_lat, sec_lng = rectangles_viewports[63][1]
viewports_2, k = create_viewports(m, sec_lat, sec_lng, 6, 6, delta, k, 'green')
rectangles_viewports.extend(viewports_2)

third_lat, third_lng = rectangles_viewports[27][1]
viewports_3, k = create_viewports(m, third_lat, third_lng, 6, 5, delta, k, 'red')
rectangles_viewports.extend(viewports_3)

fourth_lat, fourth_lng = rectangles_viewports[5][0]
viewports_4, k = create_viewports(m, fourth_lat, fourth_lng, 2, 5, -delta, k,'orange')
rectangles_viewports.extend(viewports_4)

fifth_lat, fifth_lng = rectangles_viewports[115][1]
viewports_5, k = create_viewports(m, fifth_lat, fifth_lng, 4,3, delta, k, 'yellow')
rectangles_viewports.extend(viewports_5)

sixth_lat, sixth_lng = rectangles_viewports[63][0]
viewports_6, k = create_viewports(m, sixth_lat, sixth_lng, 6, 3, -delta, k, 'cyan')
rectangles_viewports.extend(viewports_6)

seventh_lat, seventh_lng = rectangles_viewports[69][1]
viewports_7, k = create_viewports(m, seventh_lat, seventh_lng,5,6, delta, k, 'black')
rectangles_viewports.extend(viewports_7)

m

#### Verificación de Viewports Totales

In [222]:
len(rectangles_viewports)

206

---

### Cargar API key

In [223]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

---

### Configuración de la API

Preparación de los parámetros para las consultas a Google Places API:

**Endpoint**: `places:searchText` (New Places API)

**Campos solicitados**:
- `id`: Identificador único
- `displayName`: Nombre del negocio
- `formattedAddress`: Dirección completa
- `location`: Coordenadas geográficas (lat/lng)
- `rating`: Calificación promedio
- `userRatingCount`: Número total de reseñas
- `websiteUri`: Sitio web o red social
- `primaryType`: Tipo de negocio

In [224]:
search_url = "https://places.googleapis.com/v1/places:searchText"

search_headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.location,places.rating,places.userRatingCount,places.websiteUri,places.primaryType'
}

---

### Proceso de Extracción de Datos

Búsqueda sistemática de ferreterías en todos los viewports definidos.

**Proceso:**
1. Iteración sobre cada viewport (206 áreas)
2. Consulta a la API con filtros:
   - Término de búsqueda: "ferreteria"
   - Tipo incluido: "hardware_store"
   - Restricción geográfica: rectángulo del viewport
3. Agregación de resultados
4. Guardar en .csv

In [225]:
raw_data = []

for low, high in rectangles_viewports:
    try: 
        search_json = {
            'textQuery' : 'ferreteria',
            'includedType': 'hardware_store',
            'locationRestriction' : {
                'rectangle':{
                    'low':{
                        'latitude' : low[0],
                        'longitude' : low[1]
                    },
                    'high':{
                        'latitude': high[0],
                        'longitude': high[1]
                        
                    }
                }
            }
        }
        
        response = requests.post(url = search_url, json=search_json, headers = search_headers)
        response.raise_for_status()
        
        data = response.json()
        raw_data.extend(data.get('places', []))
        
        time.sleep(0.5)
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')
        time.sleep(1)

pd.DataFrame(raw_data).to_csv('datos_raw.csv', index=False)

**Resultado**: Dataset completo exportado a `datos_raw.csv`

---

### Visualizacion del dataset

In [233]:
pd.read_csv('datos_raw.csv', index_col=0)


,formattedAddress,location,rating,websiteUri,userRatingCount,displayName,primaryType
id,,,,,,,
ChIJoS84RStzVo8RMZAc0ytcFmw,"Perif. de Mérida Lic. Manuel Berzunza S/N, 972...","{'latitude': 20.955513, 'longitude': -89.6999282}",4.2,http://www.surpesa.com.mx/,40.0,"{'text': 'SURPESA Cedis', 'languageCode': 'en'}",hardware_store
ChIJych2UCtzVo8RrIjQgPltzjk,"Perif. de Mérida Lic. Manuel Berzunza, 97312 M...","{'latitude': 20.9555047, 'longitude': -89.699936}",NaN,NaN,NaN,{'text': 'La Tienda del Plomero y el Electrici...,hardware_store
ChIJiVWqvydzVo8RwNMdR4Mi72E,"x75C, C. 34 675, Tixcacal Opichen, Sin Nombre ...","{'latitude': 20.9520469, 'longitude': -89.6897...",4.4,NaN,160.0,"{'text': 'Ferrotlapalería El Farahon Dorado', ...",hardware_store
ChIJX_mhVdhyVo8RBfz5X7rnOpk,"C. 81 511, Ampliación Tixcacal Opichen, 97249 ...","{'latitude': 20.947645899999998, 'longitude': ...",4.6,NaN,253.0,"{'text': 'Tlapalería Vizcarra', 'languageCode'...",hardware_store
ChIJ4dRnBDFzVo8RZ9I0-bg8Dpc,"C. 77ᶜ 598, Ampliación Tixcacal Opichen, 97000...","{'latitude': 20.9491932, 'longitude': -89.6925...",4.9,NaN,12.0,"{'text': 'Ferrotlapalería El Foquito', 'langua...",hardware_store
...,...,...,...,...,...,...,...
ChIJAQ4vPSMNVo8RT5yOxe-nN3M,"Calle 79 732D X 92 y 94, Almendros, Cd Caucel,...","{'latitude': 20.9922643, 'longitude': -89.7090...",5.0,NaN,4.0,"{'text': 'Ferrotlapaleria ""La Marmota"" Suc Alm...",hardware_store
ChIJGwMKhIB2Vo8RSFFFgFZgbsI,"C. 38 243, Hacienda Sodzil Nte., 97115 Mérida,...","{'latitude': 21.045245299999998, 'longitude': ...",4.9,NaN,23.0,"{'text': 'Ferrotlapalería Angie y Cristhian', ...",hardware_store
ChIJEfy_U1Z1Vo8RzLXG9-q-_Kg,"Perif. de Mérida Lic. Manuel Berzunza 27, Sin ...","{'latitude': 21.0494014, 'longitude': -89.6329...",NaN,https://www.facebook.com/RefacasaManrique,NaN,"{'text': 'Manrique CEDIS', 'languageCode': 'es'}",hardware_store
